## Package Installation

In [1]:
pip install plotly==3.10.0

     |████████████████████████████████| 41.5 MB 13.0 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=24ee84691cb3ced8c14dd66a1598a7e3220e3b30a6d708a1402aeb71df731cd6
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying
  Attempting uninstall: plotly
    Found existing installation: plotly 5.4.0
    Uninstalling plotly-5.4.0:
      Successfully uninstalled plotly-5.4.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
!conda install -c conda-forge fbprophet --yes

^C


In [3]:
pip install pystan==2.19.1.1 --upgrade

     |████████████████████████████████| 67.3 MB 60 kB/s              
  Attempting uninstall: pystan
    Found existing installation: pystan 2.17.1.0
    Uninstalling pystan-2.17.1.0:
      Successfully uninstalled pystan-2.17.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.
fbprophet 0.7.1 requires setuptools-git>=1.2, which is not installed.
Note: you may need to restart the kernel to use updated packages.


## Importing Data

In [43]:
import boto3
import pandas as pd
from fbprophet import Prophet
import pystan

s3 = boto3.client('s3')
obj = s3.get_object(Bucket='dlk-cloud-tier-10-preprocessed-ml-dev', Key='sales_stockpred/pilot_phase2/history_data_2019_2021/p2_history_2019_2021.csv')
dfh = pd.read_csv(obj['Body'])
dfh.columns = ['profile_id','date','refill','rd','opening_balance']

obj = s3.get_object(Bucket='dlk-cloud-tier-10-preprocessed-ml-dev', Key='sales_stockpred/pilot_phase2/p2_2022.csv')
dfc = pd.read_csv(obj['Body'])
dfc.columns = ['profile_id','date','refill','rd','opening_balance']

df = dfh.append(dfc,ignore_index=True)
rf = df[["profile_id","date","refill"]]
rf['date'] = pd.to_datetime(rf['date'])
print("Input Dataframe Ready")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning:

Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/



Input Dataframe Ready


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## PREPROCESSING

In [44]:
import pyarrow.parquet as pq
import pandas as pd

path = 's3://dlk-cloud-tier-13-featurestore-ml-dev/Holiday-season-and-Observance-feature-group/120582440665/sagemaker/ap-southeast-1/offline-store/Holiday-season-and-Observance-feature-group-1637125134/data/year=2021/month=11/day=17/hour=04/'
table = pq.read_table(path)
data= table.to_pandas()

hol_cal=data[['date','reason','type']]
hol_cal= hol_cal.rename(columns = {'date': 'ds', 'reason': 'holiday'}, inplace = False)
hol_cal['ds'] = pd.to_datetime(hol_cal['ds'])
hol_cal=hol_cal[hol_cal['type'] == "Public Holiday"]
hol_cal.drop('type',axis='columns', inplace=True)

hol_cal['poya'] = hol_cal['holiday'].str.contains('poya',na=False,case=False)
hol_cal['vesak'] = hol_cal['holiday'].str.contains('vesak',na=False,case=False)
hol_cal['poson'] = hol_cal['holiday'].str.contains('poson',na=False,case=False)
hol_cal['national_day'] = hol_cal['holiday'].str.contains('national day',na=False,case=False)
hol_cal['avurudu'] = hol_cal['holiday'].str.contains('sinhala and tamil',na=False,case=False)
hol_cal['mayday'] = hol_cal['holiday'].str.contains('may day',na=False,case=False)
hol_cal['thaipongal'] = hol_cal['holiday'].str.contains('pongal',na=False,case=False)
hol_cal['milad_un_nabi'] = hol_cal['holiday'].str.contains('nabi',na=False,case=False)
hol_cal['christmas'] = hol_cal['holiday'].str.contains('christmas',na=False,case=False)
hol_cal['deepavali'] = hol_cal['holiday'].str.contains('deepavali',na=False,case=False)
hol_cal['hadji'] = hol_cal['holiday'].str.contains('eid al-adha',na=False,case=False)
hol_cal['ramazan'] = hol_cal['holiday'].str.contains('eid al-fitr',na=False,case=False)
hol_cal['good_friday'] = hol_cal['holiday'].str.contains('good friday',na=False,case=False)
hol_cal['mahasivarathri'] = hol_cal['holiday'].str.contains('mahasivarathri',na=False,case=False)
hol_cal['public'] = hol_cal['holiday'].str.contains('special holiday',na=False,case=False)

poya = pd.DataFrame({'holiday': 'Poya', 'ds':hol_cal[hol_cal['poya']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
vesak = pd.DataFrame({'holiday': 'Vesak', 'ds':hol_cal[hol_cal['vesak']==True]['ds'], 'lower_window': -1, 'upper_window': 2})
poson = pd.DataFrame({'holiday': 'Poson', 'ds':hol_cal[hol_cal['poson']==True]['ds'], 'lower_window': -1, 'upper_window': 2})
nationalday = pd.DataFrame({'holiday': 'National Day', 'ds':hol_cal[hol_cal['national_day']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
avurudu = pd.DataFrame({'holiday': 'Avurudu', 'ds':hol_cal[hol_cal['avurudu']==True]['ds'], 'lower_window': -3, 'upper_window': 3})
mayday = pd.DataFrame({'holiday': 'May Day', 'ds':hol_cal[hol_cal['mayday']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
thaipongal = pd.DataFrame({'holiday': 'Thaipongal', 'ds':hol_cal[hol_cal['thaipongal']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
miladunnabi = pd.DataFrame({'holiday': 'Milad Un Nabi', 'ds':hol_cal[hol_cal['milad_un_nabi']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
nationalday = pd.DataFrame({'holiday': 'National Day', 'ds':hol_cal[hol_cal['national_day']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
christmas = pd.DataFrame({'holiday': 'Christmas', 'ds':hol_cal[hol_cal['christmas']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
deepavali = pd.DataFrame({'holiday': 'Deepavali', 'ds':hol_cal[hol_cal['deepavali']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
hadji = pd.DataFrame({'holiday': 'Hadji', 'ds':hol_cal[hol_cal['hadji']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
ramazan = pd.DataFrame({'holiday': 'Ramazan', 'ds':hol_cal[hol_cal['ramazan']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
goodfriday = pd.DataFrame({'holiday': 'Good Friday', 'ds':hol_cal[hol_cal['good_friday']==True]['ds'], 'lower_window': 0, 'upper_window': 1})
mahasivarathri = pd.DataFrame({'holiday': 'Mahasivarathri', 'ds':hol_cal[hol_cal['mahasivarathri']==True]['ds'], 'lower_window': 0, 'upper_window': 1})

lockdown = pd.DataFrame({
'holiday': 'Lock_down',
  'ds': pd.to_datetime(['2021-08-20', '2021-08-21', '2021-08-22', '2021-08-23',
                        '2021-08-24', '2021-08-25', '2021-08-26', '2021-08-27',
                        '2021-08-28', '2021-08-29', '2021-08-30', '2021-08-31',
                        '2021-09-01', '2021-09-02', '2021-09-03',
                        '2021-09-04', '2021-09-05', '2021-09-06', '2021-09-07',
                        '2021-09-08', '2021-09-09', '2021-09-10', '2021-09-11',
                        '2021-09-12', '2021-09-13', '2021-09-14', '2021-09-15',
                        '2021-09-16', '2021-09-17', '2021-09-18', '2021-09-19',
                        '2021-09-20', '2021-09-21', '2021-09-22', '2021-09-23',
                        '2021-09-24', '2021-09-25', '2021-09-26', '2021-09-27',
                        '2021-09-28', '2021-09-29', '2021-09-30']),
  'lower_window': -1,
  'upper_window': 1,
      })

travel_restrictions =  pd.DataFrame({
'holiday': 'travel_restrictions',
  'ds': pd.to_datetime(['2021-07-15', '2021-07-16', '2021-07-17', '2021-07-18',
                        '2021-07-19', '2021-07-20', '2021-07-21', '2021-07-22',
                        '2021-07-23', '2021-07-24', '2021-07-25', '2021-07-26',
                        '2021-07-27', '2021-07-28', '2021-07-29', '2021-07-30',
                        '2021-07-31', '2021-08-01', '2021-08-02', '2021-08-03',
                        '2021-08-04', '2021-08-05', '2021-08-06', '2021-08-07',
                        '2021-08-08', '2021-08-09', '2021-08-10', '2021-08-11',
                        '2021-08-12', '2021-08-13', '2021-08-14', '2021-08-15',
                        '2021-08-16', '2021-08-17', '2021-08-18', '2021-08-19',
                        '2021-08-20', '2021-08-21', '2021-08-22', '2021-08-23',
                        '2021-08-24', '2021-08-25', '2021-08-26', '2021-08-27',
                        '2021-08-28', '2021-08-29', '2021-08-30', '2021-08-31',
                        '2021-09-01', '2021-09-02', '2021-09-03',
                        '2021-09-04', '2021-09-05', '2021-09-06', '2021-09-07',
                        '2021-09-08', '2021-09-09', '2021-09-10', '2021-09-11',
                        '2021-09-12', '2021-09-13', '2021-09-14', '2021-09-15',
                        '2021-09-16', '2021-09-17', '2021-09-18', '2021-09-19',
                        '2021-09-20', '2021-09-21', '2021-09-22', '2021-09-23',
                        '2021-09-24', '2021-09-25', '2021-09-26', '2021-09-27',
                        '2021-09-28', '2021-09-29', '2021-09-30', 
                        '2021-10-01', '2021-10-02', '2021-10-03',
                        '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07',
                        '2021-10-08', '2021-10-09', '2021-10-10', '2021-10-11',
                        '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15',
                        '2021-10-16', '2021-10-17', '2021-10-18', '2021-10-19',
                        '2021-10-20', '2021-10-21', '2021-10-22', '2021-10-23',
                        '2021-10-24', '2021-10-25', '2021-10-26', '2021-10-27',
                        '2021-10-28', '2021-10-29', '2021-10-30']),
  'lower_window': -1,
  'upper_window': 1,
      })

frames = [poya,vesak,poson,nationalday,avurudu,mayday,thaipongal,miladunnabi,nationalday,christmas,deepavali,hadji,ramazan,goodfriday,mahasivarathri,lockdown,travel_restrictions]
holidays= pd.concat(frames)

print("Holidays Defined")

Holidays Defined


In [45]:
import numpy as np

#define available unique profile ID's
lines = rf['profile_id'].unique()

pp_drop = [] #dropped profile IDs from pre-processing stage
pp_rf = pd.DataFrame([]) #pre-processed data

#pre-process data for each profile ID's
for Profile_id in lines: 

    #define new data frame for each & every profile ID's
    frame = rf[rf['profile_id'] == Profile_id].copy()
    check_L2_Week = rf[rf['profile_id'] == Profile_id].sort_values('date').iloc[-14:]
  
    zeros = []
    for L2Wval  in check_L2_Week['refill'].values:
        if L2Wval == 0:
            zeros.append(L2Wval)
        else:
            pass

    if len(zeros) >= 14:
        pp_drop.append(Profile_id)
    else:
        pp_rf = pp_rf.append(frame[['profile_id','date','refill']])  
     
    print("Preprocessing Complete for Profile ID ",str(Profile_id))

print("Preprocessing Complete for All Retailers ")


Preprocessing Complete for Profile ID  19640
Preprocessing Complete for Profile ID  51394
Preprocessing Complete for Profile ID  4992
Preprocessing Complete for Profile ID  149541
Preprocessing Complete for Profile ID  8246
Preprocessing Complete for Profile ID  9024
Preprocessing Complete for Profile ID  4806
Preprocessing Complete for Profile ID  84588
Preprocessing Complete for Profile ID  1186
Preprocessing Complete for Profile ID  56075
Preprocessing Complete for Profile ID  94995
Preprocessing Complete for Profile ID  75015
Preprocessing Complete for Profile ID  8721
Preprocessing Complete for Profile ID  4797
Preprocessing Complete for Profile ID  22468
Preprocessing Complete for Profile ID  102075
Preprocessing Complete for Profile ID  108655
Preprocessing Complete for Profile ID  50905
Preprocessing Complete for Profile ID  65737
Preprocessing Complete for Profile ID  56152
Preprocessing Complete for Profile ID  29941
Preprocessing Complete for Profile ID  16431
Preprocessing 

Preprocessing Complete for Profile ID  50464
Preprocessing Complete for Profile ID  16513
Preprocessing Complete for Profile ID  100612
Preprocessing Complete for Profile ID  66398
Preprocessing Complete for Profile ID  12896
Preprocessing Complete for Profile ID  119413
Preprocessing Complete for Profile ID  41609
Preprocessing Complete for Profile ID  117730
Preprocessing Complete for Profile ID  119059
Preprocessing Complete for Profile ID  30631
Preprocessing Complete for Profile ID  1184
Preprocessing Complete for Profile ID  5158
Preprocessing Complete for Profile ID  3891
Preprocessing Complete for Profile ID  943
Preprocessing Complete for Profile ID  1183
Preprocessing Complete for Profile ID  112917
Preprocessing Complete for Profile ID  117797
Preprocessing Complete for Profile ID  104474
Preprocessing Complete for Profile ID  124996
Preprocessing Complete for Profile ID  75388
Preprocessing Complete for Profile ID  16427
Preprocessing Complete for Profile ID  112725
Preproc

Preprocessing Complete for Profile ID  155504
Preprocessing Complete for Profile ID  156786
Preprocessing Complete for Profile ID  152343
Preprocessing Complete for Profile ID  152496
Preprocessing Complete for Profile ID  143492
Preprocessing Complete for Profile ID  129436
Preprocessing Complete for Profile ID  159763
Preprocessing Complete for Profile ID  104917
Preprocessing Complete for Profile ID  159161
Preprocessing Complete for Profile ID  156504
Preprocessing Complete for Profile ID  96696
Preprocessing Complete for Profile ID  151673
Preprocessing Complete for Profile ID  156508
Preprocessing Complete for Profile ID  7960
Preprocessing Complete for Profile ID  64903
Preprocessing Complete for Profile ID  34341
Preprocessing Complete for Profile ID  10639
Preprocessing Complete for Profile ID  10975
Preprocessing Complete for Profile ID  34423
Preprocessing Complete for Profile ID  7977
Preprocessing Complete for Profile ID  74593
Preprocessing Complete for Profile ID  66405


Preprocessing Complete for Profile ID  8425
Preprocessing Complete for Profile ID  4356
Preprocessing Complete for Profile ID  73525
Preprocessing Complete for Profile ID  40104
Preprocessing Complete for Profile ID  2669
Preprocessing Complete for Profile ID  95226
Preprocessing Complete for Profile ID  100664
Preprocessing Complete for Profile ID  50903
Preprocessing Complete for Profile ID  103615
Preprocessing Complete for Profile ID  24487
Preprocessing Complete for Profile ID  31954
Preprocessing Complete for Profile ID  50677
Preprocessing Complete for Profile ID  50913
Preprocessing Complete for Profile ID  20827
Preprocessing Complete for Profile ID  8244
Preprocessing Complete for Profile ID  52172
Preprocessing Complete for Profile ID  8719
Preprocessing Complete for Profile ID  75994
Preprocessing Complete for Profile ID  28861
Preprocessing Complete for Profile ID  8322
Preprocessing Complete for Profile ID  8554
Preprocessing Complete for Profile ID  73153
Preprocessing C

Preprocessing Complete for Profile ID  39616
Preprocessing Complete for Profile ID  94304
Preprocessing Complete for Profile ID  11009
Preprocessing Complete for Profile ID  56096
Preprocessing Complete for Profile ID  52515
Preprocessing Complete for Profile ID  50688
Preprocessing Complete for Profile ID  86064
Preprocessing Complete for Profile ID  23130
Preprocessing Complete for Profile ID  11010
Preprocessing Complete for Profile ID  120083
Preprocessing Complete for Profile ID  28053
Preprocessing Complete for Profile ID  102929
Preprocessing Complete for Profile ID  80931
Preprocessing Complete for Profile ID  1176
Preprocessing Complete for Profile ID  50661
Preprocessing Complete for Profile ID  29045
Preprocessing Complete for Profile ID  120086
Preprocessing Complete for Profile ID  97252
Preprocessing Complete for Profile ID  111448
Preprocessing Complete for Profile ID  114741
Preprocessing Complete for Profile ID  7659
Preprocessing Complete for Profile ID  62738
Preproc

Preprocessing Complete for Profile ID  137778
Preprocessing Complete for Profile ID  126046
Preprocessing Complete for Profile ID  125789
Preprocessing Complete for Profile ID  114650
Preprocessing Complete for Profile ID  133885
Preprocessing Complete for Profile ID  86962
Preprocessing Complete for Profile ID  152116
Preprocessing Complete for Profile ID  148860
Preprocessing Complete for Profile ID  50360
Preprocessing Complete for Profile ID  113291
Preprocessing Complete for Profile ID  155922
Preprocessing Complete for Profile ID  139418
Preprocessing Complete for Profile ID  150776
Preprocessing Complete for Profile ID  150731
Preprocessing Complete for Profile ID  159160
Preprocessing Complete for Profile ID  129336
Preprocessing Complete for Profile ID  159760
Preprocessing Complete for Profile ID  159563
Preprocessing Complete for Profile ID  159374
Preprocessing Complete for Profile ID  160945
Preprocessing Complete for Profile ID  159567
Preprocessing Complete for Profile I

Preprocessing Complete for Profile ID  158679
Preprocessing Complete for Profile ID  101978
Preprocessing Complete for Profile ID  158681
Preprocessing Complete for Profile ID  123765
Preprocessing Complete for Profile ID  138113
Preprocessing Complete for Profile ID  132174
Preprocessing Complete for Profile ID  159307
Preprocessing Complete for Profile ID  6023
Preprocessing Complete for Profile ID  159141
Preprocessing Complete for Profile ID  158673
Preprocessing Complete for Profile ID  113714
Preprocessing Complete for Profile ID  127838
Preprocessing Complete for Profile ID  144215
Preprocessing Complete for Profile ID  151087
Preprocessing Complete for Profile ID  136552
Preprocessing Complete for Profile ID  157116
Preprocessing Complete for Profile ID  152114
Preprocessing Complete for Profile ID  150778
Preprocessing Complete for Profile ID  163274
Preprocessing Complete for Profile ID  153753
Preprocessing Complete for Profile ID  123979
Preprocessing Complete for Profile I

In [46]:
#pp_drop
#pp_df[pp_df['profile_id'] == 65730].sort_values('date').iloc[-14:] 
#pp_rf.shape
#len(pp_drop)
#pp_rf['profile_id'].unique()

## Training and Inferencing

In [47]:
obj = s3.get_object(Bucket='dlk-cloud-tier-10-preprocessed-ml-dev', Key='sales_stockpred/pilot_phase2/visit_cycle.csv')
vc = pd.read_csv(obj['Body'])
vc.columns = ['profile_id','days']
vc['profile_id'] = vc['profile_id'].astype(int)
vc['days'] = vc['days'].astype(int)
print("visit cycle updated")

visit cycle updated


In [48]:
#Parameters for fbprophet - to be updated monthly after a hyperparameter tuning
changepoint_prior_scale = 0.55
seasonality_prior_scale = 0.77
holidays_prior_scale = 5
seasonality_mode = 'additive'
changepoint_range = 0.8
daily_seasonality = True
weekly_seasonality = True
yearly_seasonality = True
interval_width = 0.80
growth='linear'
mcmc_samples=0
alpha=0.05
uncertainty_samples=False
stan_backend=None
n_changepoints = 25
changepoints=None 
print("Model Parameters Defined")

Model Parameters Defined


In [49]:
#defining a function for model fitting and prediction
def run_prophet1(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=1, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 1 day forward prediction created")

def run_prophet2(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=2, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 2 day forward prediction created")

def run_prophet3(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=3, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 3 day forward prediction created")

def run_prophet4(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=4, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 4 day forward prediction created")


def run_prophet5(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=5, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 5 day forward prediction created")

def run_prophet6(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=6, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 6 day forward prediction created")

def run_prophet7(timeseries):
    
    #defining model
    model = Prophet(weekly_seasonality=weekly_seasonality,  
                     daily_seasonality=daily_seasonality, 
                     yearly_seasonality=yearly_seasonality,
                     changepoint_prior_scale=changepoint_prior_scale,
                     seasonality_prior_scale=seasonality_prior_scale,
                     holidays_prior_scale=holidays_prior_scale,
                     seasonality_mode=seasonality_mode,
                     interval_width=interval_width,
                     changepoint_range=changepoint_range,
                     n_changepoints=n_changepoints,
                     growth=growth,
                     changepoints=changepoints,
                     mcmc_samples=mcmc_samples,
                     uncertainty_samples=uncertainty_samples,
                     stan_backend=stan_backend,
                     holidays=holidays)
    
    #fitting the model
    model.fit(timeseries)
    future_dates =model.make_future_dataframe(periods=7, include_history=False)

    #predict with future dates
    prediction = model.predict(future_dates)  
    prediction = prediction[['ds','yhat']]

    return prediction

print("model for 7 day forward prediction created")

model for 1 day forward prediction created
model for 2 day forward prediction created
model for 3 day forward prediction created
model for 4 day forward prediction created
model for 5 day forward prediction created
model for 6 day forward prediction created
model for 7 day forward prediction created


In [50]:
pp_rf1 = pd.merge(pp_rf, vc, how="left", on=["profile_id"])
pp_rf1['days'].fillna(1, inplace = True)

In [51]:
#creating seperate list of dataframes for each retailer
lines1 = pp_rf1[pp_rf1['days'] == 1]['profile_id'].unique()
lines2 = pp_rf1[pp_rf1['days'] == 2]['profile_id'].unique()
lines3 = pp_rf1[pp_rf1['days'] == 3]['profile_id'].unique()
lines4 = pp_rf1[pp_rf1['days'] == 4]['profile_id'].unique()
lines5 = pp_rf1[pp_rf1['days'] == 5]['profile_id'].unique()
lines6 = pp_rf1[pp_rf1['days'] == 6]['profile_id'].unique()
lines7 = pp_rf1[pp_rf1['days'] == 7]['profile_id'].unique()

series1 = list()
series2 = list()
series3 = list()
series4 = list()
series5 = list()
series6 = list()
series7 = list()

for Profile_id in lines1:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series1.append(frame)

for Profile_id in lines2:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series2.append(frame)
    
for Profile_id in lines3:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series3.append(frame)    
    
for Profile_id in lines4:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series4.append(frame)
    
for Profile_id in lines5:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series5.append(frame)
    
for Profile_id in lines6:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series6.append(frame)
    
for Profile_id in lines7:
    frame = pp_rf[pp_rf['profile_id'] == Profile_id].copy().sort_values('date')
    frame.drop('profile_id', axis=1, inplace=True)
    frame.columns = ['ds', 'y']
    series7.append(frame)
    
print("List of dataframes representing each retailer defined")

List of dataframes representing each retailer defined


In [52]:
#multiprocessing job
from tqdm import tqdm, tnrange
from multiprocessing import Pool, cpu_count

import time
start_time = time.time()

p = Pool(cpu_count())
predictions1 = list(tqdm(p.imap(run_prophet1, series1), total=len(series1)))
predictions2 = list(tqdm(p.imap(run_prophet2, series2), total=len(series2)))
predictions3 = list(tqdm(p.imap(run_prophet3, series3), total=len(series3)))
predictions4 = list(tqdm(p.imap(run_prophet4, series4), total=len(series4)))
predictions5 = list(tqdm(p.imap(run_prophet5, series5), total=len(series5)))
predictions6 = list(tqdm(p.imap(run_prophet6, series6), total=len(series6)))
predictions7 = list(tqdm(p.imap(run_prophet7, series7), total=len(series7)))
p.close()
p.join()
print("--- %s seconds ---" % (time.time() - start_time))

100%|██████████| 96/96 [00:50<00:00,  1.90it/s]
0it [00:00, ?it/s]
100%|██████████| 4/4 [00:02<00:00,  1.63it/s]
0it [00:00, ?it/s]

--- 543.5391094684601 seconds ---


In [53]:
#final dataframe of predictions
retailers1 = pd.DataFrame(lines1, columns = ['profile_id'])
retailers2 = pd.DataFrame(lines2, columns = ['profile_id'])
retailers3 = pd.DataFrame(lines3, columns = ['profile_id'])
retailers4 = pd.DataFrame(lines4, columns = ['profile_id'])
retailers5 = pd.DataFrame(lines5, columns = ['profile_id'])
retailers6 = pd.DataFrame(lines6, columns = ['profile_id'])
retailers7 = pd.DataFrame(lines7, columns = ['profile_id'])
final_predictions = pd.DataFrame([])

for i in range(len(lines1)):
    intrm_pred = predictions1[i]
    intrm_pred['profile_id'] = retailers1.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])

for i in range(len(lines2)):
    intrm_pred = predictions2[i]
    intrm_pred['profile_id'] = retailers2.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])
    
for i in range(len(lines3)):
    intrm_pred = predictions3[i]
    intrm_pred['profile_id'] = retailers3.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])

for i in range(len(lines4)):
    intrm_pred = predictions4[i]
    intrm_pred['profile_id'] = retailers4.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])
    
for i in range(len(lines5)):
    intrm_pred = predictions5[i]
    intrm_pred['profile_id'] = retailers5.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])
    
for i in range(len(lines6)):
    intrm_pred = predictions6[i]
    intrm_pred['profile_id'] = retailers6.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])
    
for i in range(len(lines7)):
    intrm_pred = predictions7[i]
    intrm_pred['profile_id'] = retailers7.iloc[i][0]
    intrm_pred = intrm_pred[['profile_id','ds','yhat']]
    final_predictions = final_predictions.append(intrm_pred[['profile_id','ds','yhat']])

print("Prediction Data Frame Ready")

Prediction Data Frame Ready


In [54]:
final_predictions


,profile_id,ds,yhat
0,51394,2022-04-27,1129.544159
0,149541,2022-04-27,7717.890345
0,9024,2022-04-27,4096.526483
0,56075,2022-04-27,9389.321580
0,94995,2022-04-27,4381.784840
...,...,...,...
1,149952,2022-04-28,609.001486
2,149952,2022-04-29,601.849321
3,149952,2022-04-30,441.264446
4,149952,2022-05-01,426.815754


## Post-Processing

In [55]:
import math

ob = df[["profile_id","date","opening_balance"]]
ob['date'] = pd.to_datetime(ob['date'])
pred_day = ob["date"].max()
ob = ob[ob['date']==pred_day]
ob = pd.merge(final_predictions, ob, how="left", on=["profile_id"])
ob

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,profile_id,ds,yhat,date,opening_balance
0,51394,2022-04-27,1129.544159,2022-04-26,19628.3400
1,149541,2022-04-27,7717.890345,2022-04-26,8922.4000
2,9024,2022-04-27,4096.526483,2022-04-26,14581.9000
3,56075,2022-04-27,9389.321580,2022-04-26,69488.0616
4,94995,2022-04-27,4381.784840,2022-04-26,293.4800
...,...,...,...,...,...
1594,149952,2022-04-28,609.001486,2022-04-26,6493.3200
1595,149952,2022-04-29,601.849321,2022-04-26,6493.3200
1596,149952,2022-04-30,441.264446,2022-04-26,6493.3200
1597,149952,2022-05-01,426.815754,2022-04-26,6493.3200


In [56]:
#converting all minus predictions to 0 and summarizing
topup_int = ob[["profile_id","ds","yhat","opening_balance"]]
topup_int.loc[ob['yhat'] < 0, 'yhat'] = 0
topup = topup_int.groupby(["profile_id"]).yhat.sum().reset_index()
topup1 = topup_int.groupby(["profile_id"]).ds.min().reset_index()
topup2 = topup_int.groupby(["profile_id"]).opening_balance.min().reset_index()
topup3 = topup_int.groupby(["profile_id"]).ds.count().reset_index()
topup = pd.merge(topup, topup1, how="left", on=["profile_id"])
topup = pd.merge(topup, topup2, how="left", on=["profile_id"])
topup = pd.merge(topup, topup3, how="left", on=["profile_id"])
topup.columns = ['profile_id','yhat','ds','opening_balance','forecast_days']

curdate_topup = topup_int[topup_int['ds']==topup_int['ds'].min()]
curdate_ret = curdate_topup[curdate_topup['yhat']-curdate_topup['opening_balance'] > 0]
curdate_ret = curdate_ret['profile_id'].unique()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
#Getting the suggested topup value
def suggested_val(op_value, yhat_value):
    if op_value > yhat_value:
        return 0
    else:
        return yhat_value - op_value

sug_val = []
for index , rows in topup.iterrows():
    sug_val.append(suggested_val(rows['opening_balance'],rows['yhat']))

topup['suggested_value'] = sug_val

#Round the suggested value upto the nearest 500
def round_values(values):
    rounded = math.ceil(values/500)*500
    return rounded

topup['yhat'] =  round(topup['yhat'],2)
topup['suggested_value'] = topup['suggested_value'].apply(lambda x : round_values(x))
topup.columns = ['profile_id','predicted_refill','date','opening_balance','forecast_days','suggested_RD']
topup

,profile_id,predicted_refill,date,opening_balance,forecast_days,suggested_RD
0,81,2059.76,2022-04-27,5272.44,2,0
1,324,1864.07,2022-04-27,22774.02,1,0
2,398,846.38,2022-04-27,7275.66,2,0
3,441,105.28,2022-04-27,570.28,1,0
4,444,1665.58,2022-04-27,10119.71,1,0
...,...,...,...,...,...,...
1010,161791,3961.04,2022-04-27,2253.03,2,2000
1011,162045,574.95,2022-04-27,2981.09,1,0
1012,162557,1149.12,2022-04-27,2622.32,2,0
1013,163037,859.45,2022-04-27,6551.08,1,0


In [58]:
alert_base_int = topup[topup['suggested_RD']!=0]
alert_base = alert_base_int[alert_base_int['profile_id'].isin(curdate_ret)] 
alert_base = alert_base[['profile_id','date','forecast_days','opening_balance','predicted_refill','suggested_RD']]
alert_base

,profile_id,date,forecast_days,opening_balance,predicted_refill,suggested_RD
6,451,2022-04-27,1,4493.28,6842.86,2500
9,454,2022-04-27,1,3139.41,3621.26,500
16,1182,2022-04-27,1,21.79,238.00,500
29,3885,2022-04-27,1,14798.69,17124.09,2500
33,4365,2022-04-27,1,3412.44,9100.57,6000
...,...,...,...,...,...,...
996,159785,2022-04-27,1,77.12,170.98,500
997,160270,2022-04-27,2,1050.95,3767.35,3000
1007,161317,2022-04-27,2,2678.58,6239.27,4000
1009,161693,2022-04-27,2,413.34,1151.84,1000


In [59]:
obj = s3.get_object(Bucket='dlk-cloud-tier-11-inference-ml-dev', Key='sales_stockpred/gc_msisdn_list/gc_msisdn_list_2.csv')
dsr_list = pd.read_csv(obj['Body'])
dsr_list.columns = ['profile_id','retailer_name','dsr_profile_id','msisdn']

df = pd.merge(alert_base,dsr_list,on=['profile_id'],how='inner')
profile_id = alert_base['profile_id']

In [63]:
profile_id
x = 161693

In [70]:
retailer_df = df[df['profile_id']==x].copy()
msisdn = '94'+str(retailer_df.iloc[0]['msisdn'])
msisdn = '94777337305'
retailer_name = retailer_df.iloc[0]['retailer_name']
suggested_value = retailer_df.iloc[0]['suggested_RD']
opening_balance = retailer_df.iloc[0]['opening_balance']
forecast_days = retailer_df.iloc[0]['forecast_days']

k = "---TEST SMS---\nLow Stock Balance Alert for next " + str(forecast_days) + " day(s)\n\nPMSID >" + str(x) + "\nName > " + str(retailer_name) + "\nCurrent Reload Bal  > " + str(round(opening_balance,2)) + "\nReq. Reload Amount till next Visit > " + str(suggested_value) 
k

'---TEST SMS---\nLow Stock Balance Alert for next 2 day(s)\n\nPMSID >161693\nName > M.S.S Computer.\nCurrent Reload Bal  > 413.34\nReq. Reload Amount till next Visit > 1000'

In [71]:
import requests

params = (
          ('destination', msisdn),
          ('q', 'be793f66d3ab2d0'),
          ('message', k),
          ('from', 'NS STK ALRT'),
      )
response = requests.get('https://richcommunication.dialog.lk/api/sms/inline/send', params=params)

In [23]:
for x in profile_id:
    retailer_df = df[df['profile_id']==x].copy()
    msisdn = '94'+str(retailer_df.iloc[0]['msisdn'])
    msisdn = '94777338042'
    retailer_name = retailer_df.iloc[0]['retailer_name']
    suggested_value = retailer_df.iloc[0]['suggested_RD']
    opening_balance = retailer_df.iloc[0]['opening_balance']
    forecast_days = retailer_df.iloc[0]['forecast_days']

IndexError: single positional indexer is out-of-bounds

In [26]:
k = "Low Stock Balance Alert for next " + str(forecast_days) + " day(s)\n\nPMSID >" + str(x) + "\nName > " + str(retailer_name) + "\nRating > 5 STAR\nCurrent Reload Bal  > " + str(opening_balance) + "\nReq. Reload Amount  > " + str(suggested_value) + "\nCurrent SIM Bal  > 0\nReq. DTV Units  > 0\nCurrent Lte Bal  > 0\nReq Lte Units  > 0\nReq. RS.50 Card Amount  > 0\nReq. RS.100 Card Amount  > 0\nReq. RS.49 Card Amount > 0\nReq. RS.99 Card Amount > 0"
k

'Low Stock Balance Alert for next 1 day(s)\n\nPMSID >451\nName > Vidushan Communication\nRating > 5 STAR\nCurrent Reload Bal  > 382.90999999993795\nReq. Reload Amount  > 4000\nCurrent SIM Bal  > 0\nReq. DTV Units  > 0\nCurrent Lte Bal  > 0\nReq Lte Units  > 0\nReq. RS.50 Card Amount  > 0\nReq. RS.100 Card Amount  > 0\nReq. RS.49 Card Amount > 0\nReq. RS.99 Card Amount > 0'

In [ ]:
    for x in profile_id:
        retailer_df = df[df['profile_id']==x].copy()
        msisdn = '94'+str(retailer_df.iloc[0]['msisdn'])
        retailer_name = retailer_df.iloc[0]['retailer_name']
        suggested_value = retailer_df.iloc[0]['suggested_RD']
        opening_balance = retailer_df.iloc[0]['opening_balance'] 

        k = "Low Stock Balance Alert\n\nPMSID >" + str(x) + "\nName > " + str(retailer_name) + "\nRating > 5 STAR\nCurrent Reload Bal  > " + str(opening_balance) + "\nReq. Reload Amount  > " + str(suggested_value) + "\nCurrent SIM Bal  > 0\nReq. DTV Units  > 0\nCurrent Lte Bal  > 0\nReq Lte Units  > 0\nReq. RS.50 Card Amount  > 0\nReq. RS.100 Card Amount  > 0\nReq. RS.49 Card Amount > 0\nReq. RS.99 Card Amount > 0"

        params = (
            ('destination', msisdn),
            ('q', 'be793f66d3ab2d0'),
            ('message', k),
            ('from', 'NS STK ALRT'),
        )

        response = requests.get('https://richcommunication.dialog.lk/api/sms/inline/send', params=params)
        


In [26]:
#filtered_df = final_predictions.loc[(final_predictions['ds'] >= '2021-11-01')
 #                    & (final_predictions['ds'] <= '2021-11-30')]
#pd.set_option('display.max_rows', None)
#filtered_df